<a href="https://colab.research.google.com/github/rudolfbob/ADL/blob/main/Tugas4_Rudolf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **5.1. Layers and Blocks**

In [4]:
!pip install d2l

     |████████████████████████████████| 82 kB 574 kB/s 
     |████████████████████████████████| 9.9 MB 11.2 MB/s 
     |████████████████████████████████| 11.2 MB 32.6 MB/s 
     |████████████████████████████████| 61 kB 6.4 MB/s 
     |████████████████████████████████| 900 kB 43.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you h

In [5]:
!pip install matplotlib==3.0.2

     |████████████████████████████████| 12.9 MB 4.6 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.1
    Uninstalling matplotlib-3.5.1:
      Successfully uninstalled matplotlib-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
plotnine 0.6.0 requires matplotlib>=3.1.1, but you have matplotlib 3.0.2 which is incompatible.
mizani 0.6.0 requires matplotlib>=3.1.1, but you have matplotlib 3.0.2 which is incompatible.
d2l 0.17.5 requires matplotlib==3.5.1, but you have matplotlib 3.0.2 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [6]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.1327, -0.0913, -0.0070, -0.2720, -0.1232,  0.1046,  0.0320,  0.3381,
         -0.1891,  0.0851],
        [ 0.2228,  0.0160, -0.0232, -0.1936, -0.0772,  0.1510,  0.0325,  0.2175,
         -0.1377, -0.0306]], grad_fn=<AddmmBackward0>)

# **5.1.1. A Custom Block**

In [7]:
class MLP(nn.Module):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Module` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # Hidden layer
        self.out = nn.Linear(256, 10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def forward(self, X):
        # Note here we use the funtional version of ReLU defined in the
        # nn.functional module.
        return self.out(F.relu(self.hidden(X)))

In [8]:
net = MLP()
net(X)

tensor([[-0.0033,  0.0005,  0.0021, -0.0972, -0.3121,  0.0177,  0.0105, -0.0725,
          0.0364, -0.0564],
        [ 0.1200,  0.0345,  0.0493, -0.1625, -0.3891,  0.0246, -0.1194, -0.0248,
          0.0624,  0.0331]], grad_fn=<AddmmBackward0>)

# **5.1.2. The Sequential Block**

In [9]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # Here, `module` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        for block in self._modules.values():
            X = block(X)
        return X

In [12]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1871, -0.2093, -0.3701, -0.1646,  0.0538, -0.1725,  0.1800,  0.0935,
         -0.1898, -0.0796],
        [-0.1492, -0.1989, -0.3586, -0.1260, -0.0478, -0.1696,  0.0702,  0.1391,
         -0.2120, -0.0413]], grad_fn=<AddmmBackward0>)

# **5.1.3. Executing Code in the Forward Propagation Function**

In [13]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # Use the created constant parameters, as well as the `relu` and `mm`
        # functions
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [15]:
net = FixedHiddenMLP()
net(X)

tensor(0.3566, grad_fn=<SumBackward0>)

In [16]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0649, grad_fn=<SumBackward0>)

# **5.2. Parameter Management**

In [17]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.1912],
        [0.2047]], grad_fn=<AddmmBackward0>)

# **5.2.1. Parameter Access**

In [18]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.2415,  0.2407,  0.2880,  0.0820, -0.2208,  0.3036, -0.2273, -0.2919]])), ('bias', tensor([0.2817]))])


# **5.2.1.1. Targeted Parameters**

In [19]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2817], requires_grad=True)
tensor([0.2817])


In [20]:
net[2].weight.grad == None

True

# **5.2.1.2. All Parameters at Once**

In [21]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [22]:
net.state_dict()['2.bias'].data

tensor([0.2817])

# **5.2.1.3. Collecting Parameters from Nested Blocks**

In [31]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # Nested here
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.0918],
        [0.0918]], grad_fn=<AddmmBackward0>)

In [32]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [33]:
rgnet[0][1][0].bias.data

tensor([ 0.0516, -0.2178, -0.0385, -0.0941,  0.0721,  0.0163,  0.2656,  0.1360])

# **5.2.2. Parameter Initialization**

# **5.2.2.1. Built-in Initialization**

In [35]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0010, -0.0069, -0.0149, -0.0025]), tensor(0.))

In [37]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [40]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.4042, -0.2256,  0.3986,  0.1870])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


# **5.2.2.2. Custom Initialization**

In [41]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000, -6.5832, -8.4135, -7.4072],
        [ 0.0000,  0.0000,  0.0000,  7.8347]], grad_fn=<SliceBackward0>)

In [43]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -4.5832, -6.4135, -5.4072])

# **5.2.3. Tied Parameters**

In [44]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# **5.3. Deferred Initialization**

# **5.3.1. Instantiating a Network**

In [45]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [46]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

In [47]:
X = tf.random.uniform((2, 20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

# **5.4. Custom Layers**

# **5.4.1. Layers without Parameters**

In [48]:
import torch
from torch import nn
from torch.nn import functional as F


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [49]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [50]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [51]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-5.5879e-09, grad_fn=<MeanBackward0>)

# **5.4.2. Layers with Parameters**

In [52]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [53]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.8117, -0.3979,  0.6903],
        [-1.1504, -0.2714, -0.7317],
        [-0.0511, -0.1592, -1.0218],
        [-0.9141,  0.6294, -2.2581],
        [ 0.5309,  0.3421, -0.6659]], requires_grad=True)

In [54]:
linear(torch.rand(2, 5))

tensor([[0.6884, 0.0000, 0.0000],
        [0.7547, 0.0000, 0.4965]])

In [59]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[5.0622],
        [6.1315]])

# **5.5. File I/O**

# **5.5.1. Loading and Saving Tensors**

In [60]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [61]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [62]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [63]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

# **5.5.2. Loading and Saving Model Parameters**

In [64]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [65]:
torch.save(net.state_dict(), 'mlp.params')

In [66]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [67]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

# **5.6. GPUs**

In [68]:
!nvidia-smi

Sat Apr 16 11:23:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    75W / 149W |    651MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **5.6.1. Computing Devices**

In [72]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [73]:
torch.cuda.device_count()

1

In [74]:
def try_gpu(i=0):  
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

# **5.6.2. Tensors and GPUs**

In [75]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

# **5.6.2.1. Storage on the GPU**

In [76]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [77]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.2871, 0.6451, 0.6103],
        [0.1614, 0.3659, 0.8014]])

# **5.6.2.2. Copying**

In [82]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [84]:
X + Z

tensor([[2., 2., 2.],
        [2., 2., 2.]], device='cuda:0')

In [86]:
Z.cuda(0) is Z

True

# **5.6.3. Neural Networks and GPUs**

In [87]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [88]:
net(X)

tensor([[1.1652],
        [1.1652]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [89]:
net[0].weight.data.device

device(type='cuda', index=0)